# Filtrado

## Mensajes Sexistas

Primero realizamos un filtrado sobre los datos de nuestro *dataset* de mensajes de sexismo; para lo cual, cargamos la informacion haciendo uso de `pandas`

In [22]:
import pandas as pd

data_path = './datasets/sexism_data.csv'

# Carga los datos en un DataFrame
sexism_data = pd.read_csv(data_path)

# Muestra las primeras filas para entender la estructura
sexism_data.head()

,id,dataset,text,toxicity,sexist,of_id
0,0,other,MENTION3481 i didn't even know random was an o...,0.118180,False,-1
1,1,other,Bottom two should've gone! #mkr,0.251850,False,-1
2,2,callme,MENTION3111 MENTION3424 ladyboner deserves so ...,0.113331,False,-1
3,3,other,She shall now be known as Sourpuss #MKR #KatAn...,0.531153,False,-1
4,4,other,Tarah W threw a bunch of women under the bus s...,0.118718,False,-1


Posterormente, haremos la selección de los datos que hayan sido clasificados como sexistas, y eliminamos las columnas de datos innecesarias

In [23]:
# Selecciona los datos del dataframe 'data' donde la columna 'sexist' es igual a True
sexism_processed_data = sexism_data[sexism_data['sexist'] == True]

# Elimina las columnas 'id', 'dataset', 'toxicity' y 'of_id' del dataframe 'processed_data'
sexism_processed_data = sexism_processed_data.drop(columns=['id', 'dataset', 'toxicity', 'of_id'])

# Reemplaza los caracteres no alfanuméricos en la columna 'text' del dataframe 'processed_data' por una cadena vacía
sexism_processed_data['text'] = sexism_processed_data['text'].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)

# Renombra la columna 'sexist' del dataframe 'processed_data' a 'class'
sexism_processed_data = sexism_processed_data.rename(columns={'sexist': 'class'})

# Asigna el valor 'sexism' a la columna 'class' del dataframe 'processed_data'
sexism_processed_data['class'] = 3#'sexism'

# Muestra las primeras filas para entender la nueva estructura
sexism_processed_data.head()

,text,class
10,All my sons have grown up with computer games ...,3
14,Women have more intuition than men,3
30,Women do not belong in politics,3
45,On the average men are more arrogant than women,3
55,A woman will never be truly fulfilled in life ...,3


Y ya que hemos confirmado que se tiene la estructura deseada, podemos guardarla en nuestro nuevo dataser unificado

In [24]:
# Define la ruta de salida del archivo CSV
all_in_one_output_path = "./datasets/dataset.csv"

# Guarda el dataframe 'processed_data' en un archivo CSV en la ruta especificada
sexism_processed_data.to_csv(all_in_one_output_path, index=False , mode='w') 

## Mensajes Clasistas

El tratamiento para los mensajes clasistas fue distinto, pues en el dataset encontrado solo se tenían los mensajes sin otro tipo de información, por lo que directamente se decidió por incluirlos en el dataset unificado tras hacer una labor sencilla de limpieza; en que se retiraron las comas que obstaculizaban el manejo de la información.

In [25]:
data_path = './datasets/clasism.csv'

# Carga los datos en un DataFrame
clasism_data = pd.read_csv(data_path)

# Muestra las primeras filas para entender la estructura
clasism_data.head()

,text
0,I don't understand how people can travel by su...
1,Public education is in decline. Luckily my kid...
2,It bothers me to have to stand in line like th...
3,The poor are only poor because they want to be...
4,If you can't afford good wine better not to dr...


Se hace además un tratamiento preventivo, con el que hacemos una limpieza sencilla de datos

In [26]:
# Reemplaza los caracteres no alfanuméricos en la columna 'text' del dataframe por una cadena vacía
clasism_data['text'] = clasism_data['text'].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)

Como indiqué anteriormente, la información incluida es únicamente el propio mensaje, por lo que debo insertar la segunda columna yo mismo, columna que corresponde a la clasificación de la información.

In [27]:
# Agregar una nueva columna llamada 'class' con el valor por defecto "clasism"
clasism_data['class'] = 1#'clasism'

# Muestra las primeras filas para entender la estructura resultante
clasism_data.head()

,text,class
0,I dont understand how people can travel by sub...,1
1,Public education is in decline Luckily my kids...,1
2,It bothers me to have to stand in line like th...,1
3,The poor are only poor because they want to be...,1
4,If you cant afford good wine better not to dri...,1


Ya que se tiene un formato de textos deseable, procedo a incluirlo dentro del dataset manufacturado por nosotros.

In [28]:
# Guarda el dataframe en el archivo CSV de la ruta especificada antes
clasism_data.to_csv(all_in_one_output_path, header=False , index=False , mode='a') 

## Mensajes Inaproppiados

Tienen un tratamiento practicamente identico al de los **Mensajes Racistas**

In [29]:
data_path = './datasets/inappropriate content.csv'

# Carga los datos en un DataFrame
inappropriate_data = pd.read_csv(data_path)

# Reemplaza los caracteres no alfanuméricos en la columna 'text' del dataframe por una cadena vacía
inappropriate_data['text'] = inappropriate_data['text'].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)

# Agregar una nueva columna llamada 'class' con el valor por defecto "clasism"
inappropriate_data['class'] = 5#'inappropriate'

# Muestra las primeras filas para entender la estructura resultante
inappropriate_data.head()

,text,class
0,The bathroom mat squelches with a suspicious m...,5
1,In the closet a pile of oncewhite towels now h...,5
2,The garden gnome leers obscenely its position ...,5
3,An old recliner in the den is crusty with the ...,5
4,The hallway wallpaper is an accidental collage...,5


Ya que he confirmado que los textos se han tratado adecuadamente, procedo a insertarlos dentro del dataset unificado.

In [30]:
# Guarda el dataframe en el archivo CSV de la ruta especificada antes
inappropriate_data.to_csv(all_in_one_output_path, header=False , index=False , mode='a') 

## Mensajes Inofensivos

Ya que se obtuvo información simple, solo se tiene el texto plano, por lo que tienen un tratamiento practicamente identico al de los **Mensajes Racistas** y los **Mensajes Inapropiados**

In [31]:
data_path = './datasets/None of the above.csv'

# Carga los datos en un DataFrame
inoffensive_data = pd.read_csv(data_path)

# Reemplaza los caracteres no alfanuméricos en la columna 'text' del dataframe por una cadena vacía
inoffensive_data['text'] = inoffensive_data['text'].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)

# Agregar una nueva columna llamada 'class' con el valor por defecto "clasism"
inoffensive_data['class'] = 6#'none of the above'

# Muestra las primeras filas para entender la estructura resultante
inoffensive_data.head()

,text,class
0,The quiet whisper of wind through a field of t...,6
1,Delving into the complex world of astrobiology...,6
2,Investing in communitybased language exchange ...,6
3,The mesmerizing sight of a murmuration of bird...,6
4,Exploring the depths of the ocean through unde...,6


Ya que parece que la informacion resultante no presenta inconvenientes de algún tipo, se puede proceder a incluir sus datos al dataframe unificado

In [32]:
# Guarda el dataframe en el archivo CSV de la ruta especificada antes
inoffensive_data.to_csv(all_in_one_output_path, header=False , index=False , mode='a') 